<a href="https://colab.research.google.com/github/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/07_Streamlit__Colab_DogCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 7. Dog vs Cat Classification streamlit app. 
## This notebook shows how to deploy an Streamlit app on top of Google Colaboratory (GPU's) using a trained model with Google's [Teachable Machine](https://teachablemachine.withgoogle.com/train).

![](https://github.com/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/Images/07.png?raw=True)

## 1) Run all and click the "**Link to web app**" (ignore the output of the last cell).
## 2) Modify the code in `app.py`


# Setup
(pip install what you need ...) 

In [ ]:
pip install keras streamlit pillow numpy

Installation of Streamlit of course!!

In [ ]:
!pip install streamlit -q

Here we use the ngrok cloud service. Don't change this!!

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

# Sometimes you have to run the cell below a couple of times to get the link to the app.
Use the output here!!

In [ ]:
print('Link to web app')
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

If you want to use your own model from Teachable Machin, just upload the `keras_model.h5` file.

In [ ]:
!wget https://github.com/napoles-uach/streamlit_apps/raw/main/Streamlit_Colab/Models/Dog_Cat.h5
!mv Dog_Cat.h5 keras_model.h5

The following is the `app.py` base code. It can be modified in the cell or in the folder to the left.

In [45]:
%%bash
cat > app.py <<EOF
import streamlit as st
from img_classification import teachable_machine_classification
import keras
from PIL import Image, ImageOps
import numpy as np
st.title("Image Classification with Google's Teachable Machine")
st.header("Dog vs Cat Classification Example")
st.write("Upload your favorite :dog:/:cat: Image for image classification as Dog or Cat")

uploaded_file = st.file_uploader("Choose a Dog/Cat jpg pic ...", type="jpg")
if uploaded_file is not None:
  image = Image.open(uploaded_file)
  st.image(image, caption='Uploaded pic.', use_column_width=True)
  st.write("")
  st.write("Classifying...")
  label = teachable_machine_classification(image, 'keras_model.h5') # Name of the model from Teachablemachine
  if label == 0:
    st.write("Is a :dog:")
  else:
    st.write("Is a :cat:")

EOF

This last cell would keep the app running. If stoped, the app woul be disconnected.

In [36]:
%%bash
cat > img_classification.py <<EOF
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


def teachable_machine_classification(img, weights_file):
    # Load the model
    model = tensorflow.keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array  # (Not sure if this is needed, but gives an error!!!)

    # run the inference
    prediction = model.predict(data)
    return np.argmax(prediction) # return position of the highest probability



EOF

In [ ]:
!streamlit run app.py